# Matrix Risk Engine - Strategy Backtest Notebook

This notebook demonstrates the backtesting workflow:
1. Load historical price data
2. Define trading strategy
3. Run backtest with transaction costs
4. Analyze performance metrics

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date

# Add project root to path
import sys
sys.path.insert(0, '..')

## 1. Setup Data

In [ ]:
from tests.stubs.stub_data_adapter import StubDataAdapter

# Create data adapter with sample data
adapter = StubDataAdapter()

np.random.seed(42)
dates = pd.date_range('2020-01-01', periods=504, freq='B')
symbols = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META']

# Seed data for each symbol
for symbol in symbols:
    returns = np.random.randn(504) * 0.02
    prices = 100 * np.cumprod(1 + returns)
    
    data = pd.DataFrame({
        'open': prices * (1 + np.random.randn(504) * 0.005),
        'high': prices * (1 + np.abs(np.random.randn(504)) * 0.01),
        'low': prices * (1 - np.abs(np.random.randn(504)) * 0.01),
        'close': prices,
        'volume': np.random.randint(1000000, 10000000, size=504),
    }, index=dates)
    
    adapter.seed_data(symbol=symbol, version='v1', data=data)

print(f"Loaded data for {len(symbols)} symbols")

## 2. Define Trading Strategy

In [ ]:
def momentum_strategy(prices: pd.DataFrame) -> pd.DataFrame:
    """12-month momentum with monthly rebalancing.
    
    Long top 40% of stocks by momentum.
    """
    # Calculate 252-day (12-month) returns
    momentum = prices.pct_change(252)
    
    # Rank stocks by momentum (percentile)
    ranks = momentum.rank(axis=1, pct=True)
    
    # Long top 40%, short bottom 40%
    signals = pd.DataFrame(0.0, index=prices.index, columns=prices.columns)
    signals[ranks >= 0.6] = 1.0  # Long top 40%
    signals[ranks <= 0.4] = -1.0  # Short bottom 40%
    
    # Normalize to equal weight within long/short
    long_count = (signals > 0).sum(axis=1)
    short_count = (signals < 0).sum(axis=1)
    
    for col in signals.columns:
        signals.loc[signals[col] > 0, col] = 1.0 / long_count[signals[col] > 0].replace(0, 1)
        signals.loc[signals[col] < 0, col] = -1.0 / short_count[signals[col] < 0].replace(0, 1)
    
    return signals.fillna(0)

print("Momentum strategy defined")

## 3. Run Backtest

In [ ]:
from src.adapters.vectorbt_adapter import VectorBTAdapter
from src.core.services.backtest_engine import BacktestEngine, BacktestRequest

# Create backtest adapter
backtest_adapter = VectorBTAdapter()

# Create backtest engine
engine = BacktestEngine(
    data_port=adapter,
    backtest_port=backtest_adapter,
    report_port=None,  # No tearsheet for now
)

# Create backtest request
request = BacktestRequest(
    universe=symbols,
    start_date=date(2020, 1, 1),
    end_date=date(2021, 12, 31),
    data_version='v1',
    signal_generator=momentum_strategy,
    rebalance_freq='monthly',
    transaction_costs={'spread_bps': 5, 'commission_bps': 2},
    generate_tearsheet=False,
)

# Run backtest
print("Running backtest...")
response = engine.run(request)
print("Backtest complete!")

## 4. Analyze Results

In [ ]:
# Display metrics summary
print("\nBacktest Results")
print("=" * 40)
for metric, value in response.metrics_summary.items():
    if 'return' in metric or 'drawdown' in metric or 'rate' in metric:
        print(f"{metric}: {value:.2%}")
    else:
        print(f"{metric}: {value:.4f}")

In [ ]:
# Plot equity curve
returns = response.result.returns
cumulative_returns = (1 + returns).cumprod()

fig, axes = plt.subplots(3, 1, figsize=(12, 10))

# Equity curve
axes[0].plot(cumulative_returns.index, cumulative_returns.values)
axes[0].set_title('Equity Curve')
axes[0].set_ylabel('Cumulative Return')
axes[0].grid(True, alpha=0.3)

# Daily returns
axes[1].bar(returns.index, returns.values, alpha=0.7, width=1)
axes[1].axhline(y=0, color='r', linestyle='-', alpha=0.3)
axes[1].set_title('Daily Returns')
axes[1].set_ylabel('Return')

# Drawdown
rolling_max = cumulative_returns.cummax()
drawdown = (cumulative_returns - rolling_max) / rolling_max
axes[2].fill_between(drawdown.index, drawdown.values, 0, alpha=0.5, color='red')
axes[2].set_title('Drawdown')
axes[2].set_ylabel('Drawdown')

plt.tight_layout()
plt.show()

## 5. Trade Analysis

In [ ]:
# Analyze trades
trades = response.result.trades

if not trades.empty:
    print(f"\nTotal Trades: {len(trades)}")
    print(f"Total Trade Value: ${trades['quantity'].abs().sum():,.0f}")
    print(f"Total Transaction Costs: ${trades['cost'].sum():,.2f}")
    
    # Trades by symbol
    trades_by_symbol = trades.groupby('symbol').agg({
        'quantity': ['count', 'sum'],
        'cost': 'sum'
    })
    print("\nTrades by Symbol:")
    print(trades_by_symbol)
else:
    print("No trades recorded")